In [91]:
import numpy as np
import math
from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, limited_depth_first, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import random
mars_map = np.load('mars_map.npy')
nr, nc = mars_map.shape
print("Shape is ({}, {})".format(nr, nc))

def position(y,x):
    return (nr-round(y/10.00165), round(x/10.00165))
def get_height(r,c):
    return mars_map[r,c]
def get_neighbors(r,c):
    #Se encuentra los 8 vecinos a
    neighbors=[]
    directions = [(dx,dy) for dx in [-1, 0, 1] for dy in [-1, 0, 1] if (dx,dy) != (0,0)]
    for dx,dy in directions:
        new_row, new_column = r+dy, c+dx
        if 0 <= new_row < nr and 0 <= new_column < nc:
            neighbors.append((new_row, new_column))
    return neighbors
x=2850
y=6400
case_1=position(y,x)
goal=position(6800,3150)

class Rover(SearchProblem):
    """
        Clase utilizada para describir el problema del Rover. Los estados son representados
        por medio de la posicion del Rover en la imagen.
    """

    def __init__(self, position, goal):
        """ Constructor de la clase para el problema del Rover.
            Inicializa el problema de acuerdo a la pocision asignada.
        """

        # Llama al constructor de su superclase SearchProblem (estado inicial = posicion de inicio).
        SearchProblem.__init__(self, position)

        # Define el estado meta = ciudad destino.
        self.goal_state = goal

    def actions(self, state):
        """
            Este método regresa una lista con las acciones legales que pueden ser ejecutadas de
            acuerdo con el estado especificado.
        """
        row, column= state
        height= get_height(row, column)
        actions = get_neighbors(row, column)
        height_limit = 0.25
        legal_actions = []
        for action in actions:
            action_r, action_c = action
            action_height = get_height(action_r, action_c)
            if (height_limit >= abs(height - action_height)) and action_height != -1:
                legal_actions.append(action)
        return legal_actions

    def result(self, state, action):
        """
            Este método regresa la acción que hizo, es decir, a donde se movio.

            state: Posición actual.
            action: Acción de traslado especificada.
        """
        return action

    def is_goal(self, state):
        """
            Este método evalua si un estado es meta.

            state: estado a ser evaluado
        """
        return state == self.goal_state

    def cost(self, state, action, action2):
        """
            Regresa el costo de aplicar la acción para trasladarse de su estado actual a un vecino.

            state: Posicion actual.
            action: trasladarse a una ciudad vecina, especificada como (ciudad vecina, distancia)
            state2: Una ciudad vecina de la ciudad origen
        """
        r1,c1=action
        r2, c2 = state
        #return 1
        return round(10.00165*abs(r2-r1)+abs(c2-c1))

    def heuristic(self, state):
        """
            Este método regresa un estimado de la distancia desde el estado a la meta.
			Utiliza una tabla de distancias en línea recta entre las ciudades.
            state: Ciudad actual.
        """
        r1,c1=self.goal_state
        r2, c2 = state
        #Se utiliza la distancia Manhatan para obtener valores enteros
        #return abs(r2-r1)+abs(c2-c1)
        #Se redondea para tener valores enteros
        return round(np.sqrt((r2-r1)**2+(c2-c1)**2)*10.00165)
# Despliega la secuencia de estados y acciones obtenidas como resultado
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de moverse a', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de moverse a', action)

            print('  ', state)
    else:
        print('Mala configuración del problema')

#---------------------------------------------------------------------------------------------------------------
#   Solución del problema con diferentes métodos
#---------------------------------------------------------------------------------------------------------------

# posibles expectadores para las búsquedas
#my_viewer = None
my_viewer = BaseViewer()       # Solo estadísticas
#my_viewer = ConsoleViewer()    # Texto en la consola

# Crea un PSA y lo resuelve mediante una búsqueda de árbol con el algoritmo
# primero en anchura

result = breadth_first(Rover(case_1,goal), graph_search=True, viewer=my_viewer)

# despliega las estadísticas de búsqueda si no se seleccionó un espectador
if my_viewer != None:
    print('Stats:')
    print(my_viewer.stats)

# despliega el resultado de la búsqueda primero en anchura

print()
print('>> Búsqueda Primero en Anchura <<')
display(result)

# resuelve el problema utilizando búsqueda de grafo con el algoritmo de
# primero en profundidad
# Se descarta, puesto que en casos que estaba a 4 turnos del estado meta, logro encontrar una solucion en 900 iteraciones.
#result = depth_first(Rover(case_1, goal), graph_search=True)
#print()
#print('>> Búsqueda Primero en Profundidad <<')
#display(result)

# resuelve el problema utilizando búsqueda de grafo con el algoritmo de
# A*.

result = astar(Rover(case_1, goal), graph_search=True)
print()
print('>> Búsqueda A* <<')
display(result)

# resuelve el problema utilizando búsqueda de grafo con el algoritmo de
# Primero en Profundidad Iterativa.
#Se pone como otro metodo de busqueda, sin embargo, se tiene que configurar lo profundo y 
# para ello, se necesita de otro metodo de busqueda para ver una profundidad adecuada.
result = limited_depth_first(Rover(case_1,goal),depth_limit = 2000,graph_search = True)
print()
print('>> Búsqueda de Primero en Profundidad Limitada <<')
display(result)

#Cada uno de los métodos imprime:
# 1. el método de la búsqueda
# 2. Donde inicia con la posición ya convertida en la escala
# 3. Describe donde está y a donde se mueve en cada iteración
# 4. Menciona cuando meta lograda y su costo total en metros

#---------------------------------------------------------------------------------------------------------------
#   Fin del archivo
#---------------------------------------------------------------------------------------------------------------

Shape is (1815, 756)
Stats:
{'max_fringe_size': 252, 'visited_nodes': 29492, 'iterations': 29492}

>> Búsqueda Primero en Anchura <<
Configuración inicial
   (1175, 285)
1 - Después de moverse a (1174, 284)
   (1174, 284)
2 - Después de moverse a (1173, 284)
   (1173, 284)
3 - Después de moverse a (1172, 283)
   (1172, 283)
4 - Después de moverse a (1171, 282)
   (1171, 282)
5 - Después de moverse a (1170, 281)
   (1170, 281)
6 - Después de moverse a (1169, 280)
   (1169, 280)
7 - Después de moverse a (1168, 279)
   (1168, 279)
8 - Después de moverse a (1167, 279)
   (1167, 279)
9 - Después de moverse a (1166, 279)
   (1166, 279)
10 - Después de moverse a (1165, 279)
   (1165, 279)
11 - Después de moverse a (1164, 279)
   (1164, 279)
12 - Después de moverse a (1163, 279)
   (1163, 279)
13 - Después de moverse a (1162, 280)
   (1162, 280)
14 - Después de moverse a (1161, 280)
   (1161, 280)
15 - Después de moverse a (1160, 281)
   (1160, 281)
16 - Después de moverse a (1159, 282)
   (11


>> Búsqueda A* <<
Configuración inicial
   (1175, 285)
1 - Después de moverse a (1174, 285)
   (1174, 285)
2 - Después de moverse a (1173, 285)
   (1173, 285)
3 - Después de moverse a (1172, 285)
   (1172, 285)
4 - Después de moverse a (1171, 285)
   (1171, 285)
5 - Después de moverse a (1170, 284)
   (1170, 284)
6 - Después de moverse a (1169, 284)
   (1169, 284)
7 - Después de moverse a (1168, 284)
   (1168, 284)
8 - Después de moverse a (1167, 284)
   (1167, 284)
9 - Después de moverse a (1166, 284)
   (1166, 284)
10 - Después de moverse a (1165, 283)
   (1165, 283)
11 - Después de moverse a (1164, 283)
   (1164, 283)
12 - Después de moverse a (1163, 283)
   (1163, 283)
13 - Después de moverse a (1162, 283)
   (1162, 283)
14 - Después de moverse a (1161, 284)
   (1161, 284)
15 - Después de moverse a (1160, 284)
   (1160, 284)
16 - Después de moverse a (1159, 284)
   (1159, 284)
17 - Después de moverse a (1158, 284)
   (1158, 284)
18 - Después de moverse a (1157, 284)
   (1157, 284)


>> Búsqueda de Primero en Profundidad Limitada <<
Configuración inicial
   (1175, 285)
1 - Después de moverse a (1176, 286)
   (1176, 286)
2 - Después de moverse a (1176, 287)
   (1176, 287)
3 - Después de moverse a (1177, 288)
   (1177, 288)
4 - Después de moverse a (1176, 289)
   (1176, 289)
5 - Después de moverse a (1176, 290)
   (1176, 290)
6 - Después de moverse a (1176, 291)
   (1176, 291)
7 - Después de moverse a (1176, 292)
   (1176, 292)
8 - Después de moverse a (1177, 292)
   (1177, 292)
9 - Después de moverse a (1178, 291)
   (1178, 291)
10 - Después de moverse a (1178, 292)
   (1178, 292)
11 - Después de moverse a (1178, 293)
   (1178, 293)
12 - Después de moverse a (1179, 292)
   (1179, 292)
13 - Después de moverse a (1179, 293)
   (1179, 293)
14 - Después de moverse a (1180, 293)
   (1180, 293)
15 - Después de moverse a (1181, 294)
   (1181, 294)
16 - Después de moverse a (1182, 294)
   (1182, 294)
17 - Después de moverse a (1183, 295)
   (1183, 295)
18 - Después de move

1137 - Después de moverse a (1079, 292)
   (1079, 292)
1138 - Después de moverse a (1078, 292)
   (1078, 292)
1139 - Después de moverse a (1077, 292)
   (1077, 292)
1140 - Después de moverse a (1076, 292)
   (1076, 292)
1141 - Después de moverse a (1075, 291)
   (1075, 291)
1142 - Después de moverse a (1074, 291)
   (1074, 291)
1143 - Después de moverse a (1073, 290)
   (1073, 290)
1144 - Después de moverse a (1072, 290)
   (1072, 290)
1145 - Después de moverse a (1071, 290)
   (1071, 290)
1146 - Después de moverse a (1070, 289)
   (1070, 289)
1147 - Después de moverse a (1069, 288)
   (1069, 288)
1148 - Después de moverse a (1070, 288)
   (1070, 288)
1149 - Después de moverse a (1071, 288)
   (1071, 288)
1150 - Después de moverse a (1072, 288)
   (1072, 288)
1151 - Después de moverse a (1073, 289)
   (1073, 289)
1152 - Después de moverse a (1074, 290)
   (1074, 290)
1153 - Después de moverse a (1075, 290)
   (1075, 290)
1154 - Después de moverse a (1074, 289)
   (1074, 289)
1155 - Des

In [90]:
# resuelve el problema utilizando búsqueda de grafo con el algoritmo de
# Greedy.

result = greedy(Rover(case_1,goal),graph_search = True)
print()
print('>> Búsqueda Greedy <<')
display(result)


>> Búsqueda Greedy <<
Configuración inicial
   (1175, 285)
1 - Después de moverse a (1174, 286)
   (1174, 286)
2 - Después de moverse a (1173, 287)
   (1173, 287)
3 - Después de moverse a (1172, 286)
   (1172, 286)
4 - Después de moverse a (1171, 286)
   (1171, 286)
5 - Después de moverse a (1171, 287)
   (1171, 287)
6 - Después de moverse a (1170, 287)
   (1170, 287)
7 - Después de moverse a (1169, 287)
   (1169, 287)
8 - Después de moverse a (1168, 286)
   (1168, 286)
9 - Después de moverse a (1167, 285)
   (1167, 285)
10 - Después de moverse a (1166, 284)
   (1166, 284)
11 - Después de moverse a (1165, 283)
   (1165, 283)
12 - Después de moverse a (1164, 283)
   (1164, 283)
13 - Después de moverse a (1163, 283)
   (1163, 283)
14 - Después de moverse a (1162, 283)
   (1162, 283)
15 - Después de moverse a (1161, 284)
   (1161, 284)
16 - Después de moverse a (1160, 285)
   (1160, 285)
17 - Después de moverse a (1159, 285)
   (1159, 285)
18 - Después de moverse a (1158, 285)
   (1158, 

In [85]:
y1,x1=7200, 4000
y2,x2=5000, 6000

case_showcase_1=position(y1,x1)
case_showcase_2=position(y2,x2)

goal_showcase_1=position(7300,4200)
goal_showcase_2=position(5200,6000)

In [52]:
#Caso 1: inicio: (7200, 4000)=> meta:(7300,4200). Distancia menor a 500 metros.
result_showcase_1 = astar(Rover(case_showcase_1,goal_showcase_1),graph_search = True)
print()
print('>> Búsqueda A* <<')
display(result_showcase_1)


>> Búsqueda de Primero en Profundidad Iterativa* <<
Configuración inicial
   (1095, 400)
1 - Después de moverse a (1096, 401)
   (1096, 401)
2 - Después de moverse a (1097, 402)
   (1097, 402)
3 - Después de moverse a (1098, 403)
   (1098, 403)
4 - Después de moverse a (1097, 404)
   (1097, 404)
5 - Después de moverse a (1098, 405)
   (1098, 405)
6 - Después de moverse a (1099, 406)
   (1099, 406)
7 - Después de moverse a (1098, 407)
   (1098, 407)
8 - Después de moverse a (1098, 408)
   (1098, 408)
9 - Después de moverse a (1099, 409)
   (1099, 409)
10 - Después de moverse a (1100, 410)
   (1100, 410)
11 - Después de moverse a (1101, 411)
   (1101, 411)
12 - Después de moverse a (1100, 411)
   (1100, 411)
13 - Después de moverse a (1101, 412)
   (1101, 412)
14 - Después de moverse a (1102, 412)
   (1102, 412)
15 - Después de moverse a (1103, 412)
   (1103, 412)
16 - Después de moverse a (1104, 412)
   (1104, 412)
17 - Después de moverse a (1105, 413)
   (1105, 413)
18 - Después de mo

In [86]:
#Caso 2: inicio: (12000, 1500)=> meta:(12200,1350). Distancia menor a 500 metros.
result_showcase_2 = astar(Rover(case_showcase_2,goal_showcase_2),graph_search = True)
print()
print('>> Búsqueda A* <<')
display(result_showcase_2)


>> Búsqueda A* <<
Configuración inicial
   (1315, 600)
1 - Después de moverse a (1314, 600)
   (1314, 600)
2 - Después de moverse a (1313, 600)
   (1313, 600)
3 - Después de moverse a (1312, 600)
   (1312, 600)
4 - Después de moverse a (1311, 601)
   (1311, 601)
5 - Después de moverse a (1310, 601)
   (1310, 601)
6 - Después de moverse a (1309, 601)
   (1309, 601)
7 - Después de moverse a (1308, 601)
   (1308, 601)
8 - Después de moverse a (1307, 601)
   (1307, 601)
9 - Después de moverse a (1306, 601)
   (1306, 601)
10 - Después de moverse a (1305, 600)
   (1305, 600)
11 - Después de moverse a (1305, 599)
   (1305, 599)
12 - Después de moverse a (1304, 599)
   (1304, 599)
13 - Después de moverse a (1303, 600)
   (1303, 600)
14 - Después de moverse a (1303, 601)
   (1303, 601)
15 - Después de moverse a (1303, 602)
   (1303, 602)
16 - Después de moverse a (1302, 602)
   (1302, 602)
17 - Después de moverse a (1302, 601)
   (1302, 601)
18 - Después de moverse a (1301, 600)
   (1301, 600)

In [54]:
y3,x3=17500, 4000
y4,x4=1000, 6000
case_showcase_3=position(y3,x3)
case_showcase_4=position(y4,x4)

goal_showcase_3=position(16500,4000)
goal_showcase_4=position(3000,6000)

In [55]:
#Caso 3: inicio: (17500, 4000)=> meta:(16500,4000). Distancia entre 1000 y 5000 metros.
result_showcase_3 = astar(Rover(case_showcase_3,goal_showcase_3),graph_search = True)
print()
print('>> Búsqueda A* <<')
display(result_showcase_3)


>> Búsqueda de Primero en Profundidad Limitada <<
Configuración inicial
   (65, 400)
1 - Después de moverse a (66, 400)
   (66, 400)
2 - Después de moverse a (67, 399)
   (67, 399)
3 - Después de moverse a (68, 400)
   (68, 400)
4 - Después de moverse a (69, 400)
   (69, 400)
5 - Después de moverse a (70, 401)
   (70, 401)
6 - Después de moverse a (71, 401)
   (71, 401)
7 - Después de moverse a (72, 400)
   (72, 400)
8 - Después de moverse a (73, 401)
   (73, 401)
9 - Después de moverse a (74, 402)
   (74, 402)
10 - Después de moverse a (74, 403)
   (74, 403)
11 - Después de moverse a (74, 404)
   (74, 404)
12 - Después de moverse a (75, 403)
   (75, 403)
13 - Después de moverse a (76, 404)
   (76, 404)
14 - Después de moverse a (77, 405)
   (77, 405)
15 - Después de moverse a (77, 406)
   (77, 406)
16 - Después de moverse a (77, 407)
   (77, 407)
17 - Después de moverse a (78, 408)
   (78, 408)
18 - Después de moverse a (79, 409)
   (79, 409)
19 - Después de moverse a (80, 410)
   (8

1259 - Después de moverse a (223, 474)
   (223, 474)
1260 - Después de moverse a (223, 473)
   (223, 473)
1261 - Después de moverse a (224, 472)
   (224, 472)
1262 - Después de moverse a (225, 473)
   (225, 473)
1263 - Después de moverse a (226, 473)
   (226, 473)
1264 - Después de moverse a (225, 472)
   (225, 472)
1265 - Después de moverse a (226, 471)
   (226, 471)
1266 - Después de moverse a (227, 471)
   (227, 471)
1267 - Después de moverse a (228, 471)
   (228, 471)
1268 - Después de moverse a (229, 470)
   (229, 470)
1269 - Después de moverse a (229, 469)
   (229, 469)
1270 - Después de moverse a (230, 468)
   (230, 468)
1271 - Después de moverse a (231, 468)
   (231, 468)
1272 - Después de moverse a (232, 469)
   (232, 469)
1273 - Después de moverse a (233, 469)
   (233, 469)
1274 - Después de moverse a (232, 468)
   (232, 468)
1275 - Después de moverse a (232, 467)
   (232, 467)
1276 - Después de moverse a (233, 466)
   (233, 466)
1277 - Después de moverse a (234, 466)
   (234

In [56]:
#Caso 4: inicio: (1000, 6000)=> meta:(3000,6000). Distancia entre 1000 y 5000 metros.
result_showcase_4 = astar(Rover(case_showcase_4,goal_showcase_4),graph_search = True)
print()
print('>> Búsqueda de Primero en Profundidad Limitada <<')
display(result_showcase_4)


>> Búsqueda de Primero en Profundidad Limitada <<
Configuración inicial
   (1715, 600)
1 - Después de moverse a (1715, 599)
   (1715, 599)
2 - Después de moverse a (1715, 598)
   (1715, 598)
3 - Después de moverse a (1715, 597)
   (1715, 597)
4 - Después de moverse a (1714, 596)
   (1714, 596)
5 - Después de moverse a (1713, 595)
   (1713, 595)
6 - Después de moverse a (1712, 594)
   (1712, 594)
7 - Después de moverse a (1712, 593)
   (1712, 593)
8 - Después de moverse a (1712, 592)
   (1712, 592)
9 - Después de moverse a (1711, 591)
   (1711, 591)
10 - Después de moverse a (1710, 590)
   (1710, 590)
11 - Después de moverse a (1710, 589)
   (1710, 589)
12 - Después de moverse a (1711, 588)
   (1711, 588)
13 - Después de moverse a (1711, 587)
   (1711, 587)
14 - Después de moverse a (1712, 588)
   (1712, 588)
15 - Después de moverse a (1712, 589)
   (1712, 589)
16 - Después de moverse a (1713, 590)
   (1713, 590)
17 - Después de moverse a (1713, 589)
   (1713, 589)
18 - Después de move

841 - Después de moverse a (1685, 596)
   (1685, 596)
842 - Después de moverse a (1685, 597)
   (1685, 597)
843 - Después de moverse a (1684, 596)
   (1684, 596)
844 - Después de moverse a (1684, 597)
   (1684, 597)
845 - Después de moverse a (1683, 596)
   (1683, 596)
846 - Después de moverse a (1683, 597)
   (1683, 597)
847 - Después de moverse a (1682, 597)
   (1682, 597)
848 - Después de moverse a (1681, 597)
   (1681, 597)
849 - Después de moverse a (1681, 598)
   (1681, 598)
850 - Después de moverse a (1681, 599)
   (1681, 599)
851 - Después de moverse a (1680, 600)
   (1680, 600)
852 - Después de moverse a (1679, 601)
   (1679, 601)
853 - Después de moverse a (1678, 602)
   (1678, 602)
854 - Después de moverse a (1677, 602)
   (1677, 602)
855 - Después de moverse a (1677, 601)
   (1677, 601)
856 - Después de moverse a (1677, 600)
   (1677, 600)
857 - Después de moverse a (1678, 599)
   (1678, 599)
858 - Después de moverse a (1679, 598)
   (1679, 598)
859 - Después de moverse a (

1632 - Después de moverse a (1583, 561)
   (1583, 561)
1633 - Después de moverse a (1582, 561)
   (1582, 561)
1634 - Después de moverse a (1583, 560)
   (1583, 560)
1635 - Después de moverse a (1584, 559)
   (1584, 559)
1636 - Después de moverse a (1584, 558)
   (1584, 558)
1637 - Después de moverse a (1583, 557)
   (1583, 557)
1638 - Después de moverse a (1582, 558)
   (1582, 558)
1639 - Después de moverse a (1581, 559)
   (1581, 559)
1640 - Después de moverse a (1582, 560)
   (1582, 560)
1641 - Después de moverse a (1581, 561)
   (1581, 561)
1642 - Después de moverse a (1581, 562)
   (1581, 562)
1643 - Después de moverse a (1582, 563)
   (1582, 563)
1644 - Después de moverse a (1583, 564)
   (1583, 564)
1645 - Después de moverse a (1584, 564)
   (1584, 564)
1646 - Después de moverse a (1585, 565)
   (1585, 565)
1647 - Después de moverse a (1586, 566)
   (1586, 566)
1648 - Después de moverse a (1587, 567)
   (1587, 567)
1649 - Después de moverse a (1588, 568)
   (1588, 568)
1650 - Des

2509 - Después de moverse a (1518, 598)
   (1518, 598)
2510 - Después de moverse a (1518, 599)
   (1518, 599)
2511 - Después de moverse a (1518, 600)
   (1518, 600)
2512 - Después de moverse a (1517, 601)
   (1517, 601)
2513 - Después de moverse a (1516, 601)
   (1516, 601)
2514 - Después de moverse a (1515, 600)
¡Meta lograda con costo = 19402 !
   (1515, 600)


In [73]:
y5,x5=5800,3500
y6,x6=6000, 3800
case_showcase_5=position(y5,x5)
case_showcase_6=position(y6,x6)

goal_showcase_5=position(13000,1000)
goal_showcase_6=position(15000,3000)

In [74]:
#Caso 5: inicio: (6000, 3500)=> meta:(17500,2000). Distancia de más de 10000 metros.
result_showcase_5 = astar(Rover(case_showcase_5,goal_showcase_5),graph_search = True)
print()
print('>> Búsqueda A* <<')
display(result_showcase_5)


>> Búsqueda A* <<
Configuración inicial
   (1235, 350)
1 - Después de moverse a (1234, 349)
   (1234, 349)
2 - Después de moverse a (1233, 348)
   (1233, 348)
3 - Después de moverse a (1232, 347)
   (1232, 347)
4 - Después de moverse a (1231, 347)
   (1231, 347)
5 - Después de moverse a (1230, 346)
   (1230, 346)
6 - Después de moverse a (1229, 346)
   (1229, 346)
7 - Después de moverse a (1228, 346)
   (1228, 346)
8 - Después de moverse a (1227, 346)
   (1227, 346)
9 - Después de moverse a (1226, 346)
   (1226, 346)
10 - Después de moverse a (1225, 345)
   (1225, 345)
11 - Después de moverse a (1224, 345)
   (1224, 345)
12 - Después de moverse a (1223, 345)
   (1223, 345)
13 - Después de moverse a (1222, 345)
   (1222, 345)
14 - Después de moverse a (1221, 345)
   (1221, 345)
15 - Después de moverse a (1220, 345)
   (1220, 345)
16 - Después de moverse a (1219, 345)
   (1219, 345)
17 - Después de moverse a (1218, 345)
   (1218, 345)
18 - Después de moverse a (1217, 345)
   (1217, 345)

1103 - Después de moverse a (437, 518)
   (437, 518)
1104 - Después de moverse a (437, 517)
   (437, 517)
1105 - Después de moverse a (436, 516)
   (436, 516)
1106 - Después de moverse a (435, 515)
   (435, 515)
1107 - Después de moverse a (434, 514)
   (434, 514)
1108 - Después de moverse a (434, 515)
   (434, 515)
1109 - Después de moverse a (434, 516)
   (434, 516)
1110 - Después de moverse a (434, 517)
   (434, 517)
1111 - Después de moverse a (433, 518)
   (433, 518)
1112 - Después de moverse a (433, 519)
   (433, 519)
1113 - Después de moverse a (432, 519)
   (432, 519)
1114 - Después de moverse a (431, 519)
   (431, 519)
1115 - Después de moverse a (430, 520)
   (430, 520)
1116 - Después de moverse a (430, 519)
   (430, 519)
1117 - Después de moverse a (430, 518)
   (430, 518)
1118 - Después de moverse a (429, 517)
   (429, 517)
1119 - Después de moverse a (429, 516)
   (429, 516)
1120 - Después de moverse a (428, 515)
   (428, 515)
1121 - Después de moverse a (428, 516)
   (428

In [66]:
#Caso 6: inicio: (6000, 4000)=> meta:(15000,3000). Distancia de más de 10000 metros.
result_showcase_6 = astar(Rover(case_showcase_6,goal_showcase_6),graph_search = True)
print()
print('>> Búsqueda A* <<')
display(result_showcase_6)


>> Búsqueda A* <<
Configuración inicial
   (1215, 400)
1 - Después de moverse a (1214, 399)
   (1214, 399)
2 - Después de moverse a (1213, 398)
   (1213, 398)
3 - Después de moverse a (1213, 397)
   (1213, 397)
4 - Después de moverse a (1212, 396)
   (1212, 396)
5 - Después de moverse a (1211, 396)
   (1211, 396)
6 - Después de moverse a (1210, 395)
   (1210, 395)
7 - Después de moverse a (1210, 394)
   (1210, 394)
8 - Después de moverse a (1210, 393)
   (1210, 393)
9 - Después de moverse a (1209, 392)
   (1209, 392)
10 - Después de moverse a (1208, 391)
   (1208, 391)
11 - Después de moverse a (1207, 390)
   (1207, 390)
12 - Después de moverse a (1206, 389)
   (1206, 389)
13 - Después de moverse a (1205, 389)
   (1205, 389)
14 - Después de moverse a (1204, 388)
   (1204, 388)
15 - Después de moverse a (1203, 387)
   (1203, 387)
16 - Después de moverse a (1203, 386)
   (1203, 386)
17 - Después de moverse a (1202, 386)
   (1202, 386)
18 - Después de moverse a (1201, 385)
   (1201, 385)

1167 - Después de moverse a (408, 473)
   (408, 473)
1168 - Después de moverse a (407, 472)
   (407, 472)
1169 - Después de moverse a (406, 471)
   (406, 471)
1170 - Después de moverse a (405, 470)
   (405, 470)
1171 - Después de moverse a (404, 469)
   (404, 469)
1172 - Después de moverse a (403, 468)
   (403, 468)
1173 - Después de moverse a (402, 467)
   (402, 467)
1174 - Después de moverse a (401, 466)
   (401, 466)
1175 - Después de moverse a (402, 466)
   (402, 466)
1176 - Después de moverse a (402, 465)
   (402, 465)
1177 - Después de moverse a (401, 464)
   (401, 464)
1178 - Después de moverse a (400, 463)
   (400, 463)
1179 - Después de moverse a (399, 462)
   (399, 462)
1180 - Después de moverse a (399, 461)
   (399, 461)
1181 - Después de moverse a (399, 460)
   (399, 460)
1182 - Después de moverse a (399, 459)
   (399, 459)
1183 - Después de moverse a (399, 458)
   (399, 458)
1184 - Después de moverse a (399, 457)
   (399, 457)
1185 - Después de moverse a (399, 456)
   (399